In [7]:
from pynq import Overlay
base = Overlay("/home/xilinx/jupyter_notebooks/MisCosas/Config/ControlMotor.bit")

from pynq.lib import AxiGPIO

import matplotlib.pyplot as plt
import time

In [27]:
AXIGPIO0=base.ip_dict['axi_gpio_0']
SP=AxiGPIO(AXIGPIO0).channel1
RPM=AxiGPIO(AXIGPIO0).channel2

AXIGPIO1=base.ip_dict['axi_gpio_1']
P=AxiGPIO(AXIGPIO1).channel1
ITs=AxiGPIO(AXIGPIO1).channel2

AXIGPIO2=base.ip_dict['axi_gpio_2']
ERROR=AxiGPIO(AXIGPIO2).channel1
CV=AxiGPIO(AXIGPIO2).channel2

In [2]:
def ToBinary(IN, NBits):
    B=[]
    while(IN>1):
        B.append(IN%2)
        IN=IN//2
    B.append(IN)
        
    while(len(B)<NBits):
        B.append(0)
        
    return B

In [3]:
def ToDecimalBinary(IN, ParteEntera):
    B=[]
    for i in range(0, ParteEntera):
        PM=IN-2**(-i-1)
        if(PM>=0): 
            B.append(1)
            IN=PM
        else: B.append(0)
            
    return B

In [4]:
def ToDecimalEntero(IN):
    DE=0
    for i in range(len(IN)-1, -1, -1):
        DE+=(2**i)*IN[i]
        
    return DE

In [5]:
def InvertBinary(IN):
    B=[]
    for i in range(len(IN)-1, -1, -1):
        B.append(IN[i])
        
    return B

In [14]:
def ComprobarRango(IN, VMax, VMin):
    if IN<VMax and IN>VMin:
        return True
    else:
        return False

In [18]:
Consigna=float(input("RPM: "))
CteP=float(input("P: "))
CteI=float(input("KI: "))

if ComprobarRango(Consigna, 500, 50) and ComprobarRango(CteP, 19, 0) and ComprobarRango(CteI, 1, 0):
    SSPP=int(Consigna*0.3)
    PP=ToDecimalEntero(InvertBinary(InvertBinary(ToBinary(CteP//1, 4))+ToDecimalBinary(CteP-CteP//1, 12)))
    PI=ToDecimalEntero(InvertBinary(InvertBinary(ToBinary(CteI//1, 1))+ToDecimalBinary(CteI-CteI//1, 15)))
    
    SP[:].write(SSPP)
    P[:].write(PP)
    ITs[:].write(PI)
else:
    print("CAGASTE")

RPM: 12
P: 12
KI: 12
CAGASTE


In [ ]:
ProcessValue=[]
ControlValue=[]
SetPoint=[]
Diferencia=[]

for i in range(0,100):
    ProcessValue.append(RPM.read())
    ControlValue.append(CV.read())
    SetPoint.append(SSPP)
    Diferencia.append(ERROR.read())
    
    sleep(0.1)